# Task 2 Generate Sparse Representations
#### Student Name: Yicheng Zhang
#### Student ID: 27699641

Date: 01/06/2018

Version: 1.0

Environment: Python 3.6.5 and Jupyter notebook

Libraries used:
* re (for regular expression, included in Anaconda Python 3.6) 
* os (for os directory, included in Anaconda Python 3.6)
* ntlk (for Natural Language Toolkit, need to be installed before using)
* itertools (creating iterators for efficient looping, included in Anaconda Python 3.6)
* numpy (statistics for numbers, included in Anaconda Python 3.6)

## 1.  Import libraries 

In [1]:
import re
import os
import nltk
from nltk.probability import *
from nltk.collocations import *
from itertools import chain
import itertools
from nltk.tokenize import RegexpTokenizer
from os.path import basename
import numpy as np

## 2. Read Txet files based on task1

### Read stop words list

In [2]:
stopwords_list = []
with open('./stopwords_en.txt') as f:
    stopwords_list = f.read().splitlines()

In [3]:
len(stopwords_list)

571

### Read txt files from task1

In [4]:
txt_file_path = './txt_files'

The word tokenization must use the following regular expression, "\w+(?:[-']\w+)?", and all the words must be converted into the lower case.

**Create a dictionary store the filename, content pair**
- - -
Format:
```python
{
    'ES2002a':"Hi,I'm.....",
    'Es2002b':"....",
    ...
}
```

In [5]:
txtdict = {}
for txtfile in os.listdir(txt_file_path):
    txtfile = os.path.join(txt_file_path, txtfile)
    if os.path.isfile(txtfile) and txtfile.endswith('.txt'): 
        base = basename(txtfile)
        name = os.path.splitext(base)[0]
        with open(txtfile) as f:
            text = f.read()
        txtdict[name] = text

**Create regular expression words token using RegexpTokenizer**

In [6]:
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?") 

**Create tokenized patents**

In [7]:
def tokenizePatent(pid):
    """
        the tokenization function is used to tokenize each patent.
        The one argument is patent_id.
        First, normalize the case.
        Then, use the regular expression tokenizer to tokenize the patent with the specified id
    """
    raw_patent = txtdict[pid].lower() 
    tokenized_patents = tokenizer.tokenize(raw_patent)
    return (pid, tokenized_patents) # return a tupel of patent_id and a list of tokens

patents_tokenized = dict(tokenizePatent(pid) for pid in txtdict.keys())

**Check text dictionary**

In [8]:
len(txtdict)

139

In [9]:
txtdict['ES2002a']

" Okay .\n Right .\n Um well this is the kick-off meeting for our our project . Um and um this is just what we're gonna be doing over the next twenty five minutes . Um so first of all , just to kind of make sure that we all know each other , I'm Laura and I'm the project manager . Do you want to introduce yourself again ?\n Mm-hmm .\n Great .\n Hi , I'm David and I'm supposed to be an industrial designer .\n Okay .\n And I'm Andrew and I'm uh our marketing\n Um I'm Craig and I'm User Interface .\n expert .\n**********\n Great . Okay . Um so we're designing a new remote control and um Oh I have to record who's here actually . So that's David , Andrew and Craig , isn't it ?\n And you all arrived on time .\n Um yeah so des uh design a new remote control . Um , as you can see it's supposed to be original , trendy and user friendly . Um so that's kind of our our brief , as it were . Um and so there are three different stages to the design . Um I'm not really sure what what you guys have alr

## 3. Text pre-processing

**Filter the token with stopwords list**

In [10]:
text_filtered = {}
for filename in patents_tokenized.keys():
    text_filtered[filename] = [w for w in patents_tokenized[filename] if w not in stopwords_list]

**Check the token's information**

In [11]:
words = list(chain.from_iterable(text_filtered.values()))
vocab = set(words)
print ("Vocabulary size: ",len(vocab))
print ("Total number of tokens: ", len(words))
print ("Total number of articles:", len(text_filtered))
lens = [len(value) for value in text_filtered.values()]
print ("Average document length:", np.mean(lens))
print ("Maximun document length:", np.max(lens))
print ("Minimun document length:", np.min(lens))
print ("Standard deviation of document length:", np.std(lens))

Vocabulary size:  10507
Total number of tokens:  226313
Total number of articles: 139
Average document length: 1628.1510791366907
Maximun document length: 3193
Minimun document length: 251
Standard deviation of document length: 655.1376912079487


**Find the words with highest Frequency (appear in 132 documents or more)**

In [12]:
words_2 = list(chain.from_iterable([set(value) for value in text_filtered.values()])) # set for nonredundant
fd_2 = FreqDist(words_2) # Frequency distribution
fd_2.most_common(40) # 40 words with largest numbers of frequency among documents

[('yeah', 139),
 ('um', 139),
 ('uh', 139),
 ('good', 138),
 ('mm', 138),
 ('make', 138),
 ('meeting', 137),
 ('design', 137),
 ('mm-hmm', 137),
 ('things', 136),
 ('thing', 135),
 ('remote', 135),
 ('control', 134),
 ('bit', 133),
 ('people', 133),
 ('work', 133),
 ('put', 132),
 ('kind', 130),
 ('idea', 129),
 ('time', 129),
 ('buttons', 127),
 ('lot', 126),
 ('gonna', 124),
 ('user', 124),
 ('button', 124),
 ('find', 123),
 ('hmm', 122),
 ('point', 122),
 ('minutes', 122),
 ('interface', 120),
 ('big', 120),
 ('back', 120),
 ('t_v_', 119),
 ('easy', 119),
 ('kay', 118),
 ('project', 118),
 ('guess', 117),
 ('marketing', 116),
 ('controls', 116),
 ('nice', 115)]

In [13]:
frequencyDist = fd_2.most_common(40)

In [14]:
mostFrequency = []
for words in frequencyDist:
    if words[1] > 132:
        mostFrequency.append(words[0]) # words appear more than 132 documents

In [15]:
mostFrequency

['yeah',
 'um',
 'uh',
 'good',
 'mm',
 'make',
 'meeting',
 'design',
 'mm-hmm',
 'things',
 'thing',
 'remote',
 'control',
 'bit',
 'people',
 'work']

**Filter the token with highest frequency words**

In [16]:
text_filtered_2 = {}
for filename in patents_tokenized.keys():
    text_filtered_2[filename] = [w for w in text_filtered[filename] if w not in mostFrequency]

**Check the information**

In [17]:
words = list(chain.from_iterable(text_filtered_2.values()))
vocab = set(words)
print ("Vocabulary size: ",len(vocab))
print ("Total number of tokens: ", len(words))
print ("Total number of articles:", len(text_filtered_2))
lens = [len(value) for value in text_filtered_2.values()]
print ("Average document length:", np.mean(lens))
print ("Maximun document length:", np.max(lens))
print ("Minimun document length:", np.min(lens))
print ("Standard deviation of document length:", np.std(lens))

Vocabulary size:  10491
Total number of tokens:  155740
Total number of articles: 139
Average document length: 1120.431654676259
Maximun document length: 2252
Minimun document length: 158
Standard deviation of document length: 449.4661589738761


**Remove the words length less than 3**

In [18]:
text_filtered_3 = {}
for filename in patents_tokenized.keys():
    text_filtered_3[filename] = [w for w in text_filtered_2[filename] if len(w)>= 3 ]

In [19]:
words = list(chain.from_iterable(text_filtered_3.values()))
vocab = set(words)
print ("Vocabulary size: ",len(vocab))
print ("Total number of tokens: ", len(words))
print ("Total number of articles:", len(text_filtered_3))
lens = [len(value) for value in text_filtered_3.values()]
print ("Average document length:", np.mean(lens))
print ("Maximun document length:", np.max(lens))
print ("Minimun document length:", np.min(lens))
print ("Standard deviation of document length:", np.std(lens))

Vocabulary size:  10305
Total number of tokens:  152880
Total number of articles: 139
Average document length: 1099.8561151079136
Maximun document length: 2204
Minimun document length: 157
Standard deviation of document length: 440.3174224744552


### Create the vocabulary list and save to file
>vocab.txt: It contains the unigram vocabulary in the following format, word_string:integer_index. Words in the vocabulary must be sorted in alphabetic order. For example, "absolute:22" in the following figure means that the 23rd word in the vocabulary is "absolute"
- - -
The output file should have the format:
```
a-hold:0
a_a_:1
a_a_s:2
```

In [20]:
wordlist = set(chain.from_iterable([set(value) for value in text_filtered_3.values()]))

In [21]:
result = []
for item in wordlist:
    result.append(item)

In [22]:
result.sort()

In [23]:
result[0:10]

['a-hold',
 'a_a_',
 'a_a_s',
 'a_m_i_',
 'a_n_',
 'a_p_o_g_e_e_',
 'a_s',
 'a_s_r_',
 'a_v_',
 'abandon']

In [24]:
len(result)

10305

In [25]:
with open("./vocab.txt", "w") as output:
    for index,value in enumerate(result):
        line = value + ':'+ str(index)+'\n'
        output.write(line)
output.close()

### Create topic boundaries boolean vectors for files 
> topic_seg.txt: It contains the topic boundaries encoded in boolean vectors. For example, if a meeting transcript, "ES2018d.txt" contains 10 paragraphs in total after being preprocessed, and there are topic boundaries after the 2nd, 5th, and 7th paragraphs, the boolean vector must be "ES2018d:0,1,0,0,1,0,1,0,0,1". Every line in topic_seg.txt corresponds to one meeting transcript.
- - -
Subtopics are divided by `**********`, this symbol should be the topic boundaries, however, the symbol should inside the topic above.

The output format for this file should be:
```
ES2002a:0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
```

In [26]:
topicseg = {}
for filename in txtdict.keys():
    segment = []
    for lines in txtdict[filename].split('\n'):
        if segment == '':
            segment.append('0')
        if lines != '':
            if lines == '**********':
                segment.pop()
                segment.append('1')
            else:
                segment.append('0') 
    
    topicseg[filename] = segment

**Sort file name using list (dictionary cannot be directly sorted)**

In [27]:
topicseg_file = list(topicseg.keys())
topicseg_file.sort()
topicseg_file[0:10]

['ES2002a',
 'ES2002b',
 'ES2002c',
 'ES2002d',
 'ES2003a',
 'ES2003b',
 'ES2003c',
 'ES2003d',
 'ES2004a',
 'ES2004b']

**Save the boolean vector to the file**

In [28]:
with open("./topic_segs.txt", "w") as output:
    for file in topicseg_file:
        sequence = ''
        for index in range(len(topicseg[file])):
            if index == len(topicseg[file])-1: # Last item with no comma
                sequence += str(topicseg[file][index])
            else:
                sequence += str(topicseg[file][index]) + ',' 
            
        line = file + ':'+ sequence +'\n'
        output.write(line)
output.close()

In [29]:
#Remove after using, In case inconsistent of naming style
with open("./topic_seg.txt", "w") as output:
    for file in topicseg_file:
        sequence = ''
        for index in range(len(topicseg[file])):
            if index == len(topicseg[file])-1:
                sequence += str(topicseg[file][index])
            else:
                sequence += str(topicseg[file][index]) + ',' 
            
        line = file + ':'+ sequence +'\n'
        output.write(line)
output.close()

### Create sparse files files 
> ./sparse_files/*.txt : Each txt file in the "sparse_files" folder corresponds to one of the meeting transcripts in the "txt_files" folder, and they have the same file name.  For example, "./sparse_files/ES2002a.txt" corresponds to "./txt_files/ES2002a.txt". Each file in "/sparse_files" contains the sparse representations for all its paragraphs as sample sparse file where 

> 1) each line is a paragraph and the order of the lines must match the paragraph order in the corresponding meeting transcript. 

> 2) the integer before ":" is the word index in the vocabulary and the one after is the frequency of the word in the corresponding paragraph; 

> 3) empty paragraphs after preprocessing must be excluded.

This steps is very important for text preprocessing, according to the literature:
> The pre-processed text needs to be saved in a proper format so that it can be easily used by the downstream analysis algorithm. There are a couple of ways of dumping the pre-processed text data into txt files

Given the vocabulary, each document can be represented as a sequence of integers that correspond to the tokens,
or in the following sparse form:
```
    word_index:word count
```
The text itself can be transfer like:
```
297:1
8181:1,8794:1,4141:1,4381:1,2593:2,3820:1,7414:1,10380:1
6791:1
```

In [30]:
len(result) # Checking

10305

**Create the vocabulary dictionary for faster access**
- - -
Format:
```python
{
    'foo':123,
    'bar':124
}
```

In [31]:
vocbdict = {}
for key,val in enumerate(result):
    vocbdict[val] = key

In [32]:
len(vocbdict)#Checking

10305

**Define the function sparse files**

In [33]:
def sparse_files(filename):
    path = './sparse_files/' + filename + '.txt'
    with open(path,"w") as f:      
        for lines in txtdict[filename].split('\n'):
            lines = lines.lower()
            text = ''
            lines_split = re.findall("\w+(?:[-']\w+)?",lines) # split words using right regular expression
            dic = FreqDist(lines_split) # words frequency in that line
            wordlist = []
            for item in lines_split:   # manually create a set with order
                if item not in wordlist:
                    wordlist.append(item)
            for words in wordlist:
                if words in vocbdict.keys():
                    index = str(vocbdict[words]) # find the index in vocbulary dictionary
                    if text == '': # initial the string sequence
                        text = index + ':' + str(dic[words]) # dic[words] retrieve count of this word
                    else:
                        text += ',' + index + ':' + str(dic[words])

            if text != '':
                text += '\n'
                f.write(text) # If there is something in the text
    f.close()  

**Save them to files**

In [35]:
for file in txtdict: # traverse dictionary's key
    sparse_files(file)

## 4. References

1. [NLTK](http://www.nltk.org/book/)
2. [Python os](https://docs.python.org/3/library/os.html)
3. [Tutorial Material](https://moodle.vle.monash.edu/course/view.php?id=42906&section=17#17)
4. [Tutorial Material2](https://moodle.vle.monash.edu/course/view.php?id=42906&section=16#16)
4. [Python data structure](https://docs.python.org/3/tutorial/datastructures.html)
5. [Python IO for files](https://docs.python.org/3/library/io.html?highlight=io#module-io)